# **라이브러리 로드**

In [ ]:
import pandas as pd
import numpy as np
import tarfile
import pickle
import os


# 데이터 전처리 패키지
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder

#모델 패키지
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.layers import Dropout

# 시각화 패키지
from matplotlib.colors import ListedColormap, BoundaryNorm
import matplotlib.patches as mpatches
import matplotlib.pyplot as plt
import seaborn as sns

# **차원 축소 - 변수 추출법**

## **1. 주성분 분석(PCA)**

### **데이터 불러오기**

breast-cancer.csv
- 유방 종양의 30개 물리적 특성 관련 지표
- 목표 변수 : Diagnosis(악성M/양성B)

In [ ]:
# Breast cancer dataset
data = pd.read_csv("../data/breast-cancer.csv")
X_canc = data.drop(columns=['id', 'diagnosis', 'Unnamed: 32'], errors='ignore')
y_canc = data['diagnosis']

### **시각화 함수**

In [ ]:
def plot_labelled_scatter(X, y, class_labels, s):
    le = LabelEncoder()  # 라벨 데이터를 숫자로 변환하는 인코더 초기화
    y_encoded = le.fit_transform(y)  # 클래스 레이블('M', 'B' 등)을 숫자로 변환
    num_labels = len(class_labels)  # 클래스 레이블의 총 개수

    # 데이터의 x축과 y축 범위 설정
    x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
    y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1

    # 마커와 색상 설정
    marker_array = ['o', '^', '*']  # 각 클래스의 마커 스타일
    color_array = ['#FFFF00', '#00AAFF', '#000000', '#FF00AA']  # 각 클래스의 색상
    cmap_bold = ListedColormap(color_array)  # 강한 색상을 위한 컬러맵 생성
    bnorm = BoundaryNorm(np.arange(0, num_labels + 1, 1), ncolors=num_labels)  # 클래스 경계를 위한 색상 정규화

    plt.figure(figsize=s)  # 그래프 크기 설정

    # 산점도 그리기
    plt.scatter(X[:, 0], X[:, 1], s=80, c=y_encoded, cmap=cmap_bold, norm=bnorm,
                alpha=0.4, edgecolor='black', lw=1)  # 점 크기, 색상, 투명도 및 테두리 설정

    # 그래프의 상단과 오른쪽 축 숨기기
    sp = plt.gca().spines  # 현재 축의 경계를 가져옴
    sp['top'].set_visible(False)  # 상단 경계 숨기기
    sp['right'].set_visible(False)  # 오른쪽 경계 숨기기

    plt.grid(which='both', color='lightslategrey', alpha=0.3)  # 격자 표시 설정

    # x축과 y축 범위 설정
    plt.xlim(x_min, x_max)
    plt.ylim(y_min, y_max)

    # 범례 추가
    h = []  # 범례 핸들 리스트
    for c in range(0, num_labels):
        h.append(mpatches.Patch(color=color_array[c], label=class_labels[c]))  # 각 클래스의 색상과 레이블 추가
    plt.legend(handles=h, fontsize=15, frameon=False)  # 범례 스타일 설정

### **1-1. PCA 적용**

In [ ]:
canc_norm = StandardScaler().fit(X_canc).transform(X_canc)  # 데이터를 평균 0, 분산 1로 표준화하여 PCA 적용 전에 정규화

pca = PCA(n_components=2).fit(canc_norm)  # PCA(주성분 분석)를 사용하여 데이터의 주성분 2개를 추출하도록 모델을 학습

canc_pca = pca.transform(canc_norm)  # 학습된 PCA 모델로 데이터를 변환하여 2개의 주성분으로 차원을 축소

print('Number of Features in Breast Cancer DataSet Before PCA : {}\n\nNumber of Features in Breast Cancer DataSet After PCA : {}'
      .format(X_canc.shape[1], canc_pca.shape[1]))  # PCA 전후의 데이터 특징 수 출력 (PCA 적용 전의 특징 수와 축소 후의 특징 수 비교)

### **1-2 PCA 결과 시각화**

In [ ]:
plot_labelled_scatter(canc_pca, y_canc, ['M', 'B'],(15,9)) 

plt.xlabel('First principal component',fontsize=15)
plt.ylabel('Second principal component',fontsize=15)
plt.title('Breast Cancer Dataset PCA (n_components = 2)',fontsize=17)

### **1-3 PCA 결과와 기존 feature의 상관관계 확인**

In [ ]:
fig = plt.figure(figsize=(20,9))
plt.imshow(pca.components_, interpolation = 'none', cmap = 'viridis')
feature_names = list(X_canc.columns)

plt.gca().set_xticks(np.arange(len(feature_names)));
plt.gca().set_yticks(np.arange(2));
plt.gca().set_xticklabels(feature_names, rotation=90, fontsize=14);
plt.gca().set_yticklabels(['Dimension 1', 'Dimension 2'], fontsize=14);

plt.colorbar(orientation='horizontal', ticks=[pca.components_.min(), 0, 
                                              pca.components_.max()], pad=0.5);

***

## **2. 이미지 데이터에서의 PCA 적용**
#### data : cifar10

### **데이터 로드**

In [ ]:
# CIFAR-10 tar.gz 파일의 경로를 정의
cifar10_tar_path = './data/cifar-10-python.tar.gz'

# tar.gz 파일 추출
with tarfile.open(cifar10_tar_path, 'r:gz') as tar:
    tar.extractall()  # 파일을 현재 디렉토리로 추출
    print("Files extracted.")  # 파일이 추출되었음을 출력

# 배치 데이터를 파일에서 로드하는 함수 정의
def load_batch(file_path):
    with open(file_path, 'rb') as f:
        data_dict = pickle.load(f, encoding='bytes')  # pickle을 사용하여 파일 로드
        # 이미지 데이터와 레이블 추출
        data = data_dict[b'data']  # 이미지 데이터
        labels = data_dict[b'labels']  # 레이블 데이터
        # 데이터를 이미지 형식(N, H, W, C)으로 변환 및 재배치
        data = data.reshape(-1, 3, 32, 32).transpose(0, 2, 3, 1)
        labels = np.array(labels)  # 레이블을 NumPy 배열로 변환
    return data, labels  # 변환된 이미지와 레이블 반환

# CIFAR-10 파일이 추출된 디렉토리
cifar10_dir = 'cifar-10-batches-py'

# 훈련 데이터 로드
x_train = []  # 훈련 이미지 데이터를 저장할 리스트
y_train = []  # 훈련 레이블 데이터를 저장할 리스트
for i in range(1, 6):  # 1번 배치부터 5번 배치까지 반복
    batch_file = os.path.join(cifar10_dir, f'data_batch_{i}')  # 배치 파일 경로
    data, labels = load_batch(batch_file)  # 배치 데이터 로드
    x_train.append(data)  # 이미지 데이터를 리스트에 추가
    y_train.append(labels)  # 레이블 데이터를 리스트에 추가

# 모든 훈련 배치를 연결하여 단일 배열로 변환
x_train = np.concatenate(x_train)  # 훈련 이미지 데이터 연결
y_train = np.concatenate(y_train)  # 훈련 레이블 데이터 연결

# 테스트 데이터 로드
test_batch_file = os.path.join(cifar10_dir, 'test_batch')  # 테스트 배치 파일 경로
x_test, y_test = load_batch(test_batch_file)  # 테스트 데이터 로드

### **데이터 확인**

In [ ]:
#데이터 분리
print('Traning data shape:', x_train.shape)
print('Testing data shape:', x_test.shape)

In [ ]:
y_train.shape,y_test.shape

In [ ]:
classes = np.unique(y_train)
nClasses = len(classes)
print('Total number of outputs : ', nClasses)
print('Output classes : ', classes)

In [ ]:
# 라벨 사전 정의
label_dict = {
 0: 'airplane',
 1: 'automobile',
 2: 'bird',
 3: 'cat',
 4: 'deer',
 5: 'dog',
 6: 'frog',
 7: 'horse',
 8: 'ship',
 9: 'truck',
}

In [ ]:
# 데이터 확인을 위한 시각화
plt.figure(figsize=[5,5])  # 그래프 크기 설정

# 훈련 데이터의 첫 번째 이미지 출력
plt.subplot(121)  # 첫 번째 그래프 (1행 2열 중 첫 번째 위치)
curr_img = np.reshape(x_train[0], (32, 32, 3))  # 첫 번째 훈련 데이터를 (32, 32, 3) 형식으로 재구성
plt.imshow(curr_img)  # 이미지를 표시
print(plt.title("(Label: " + str(label_dict[y_train[0]]) + ")"))  # 레이블 출력

# 테스트 데이터의 첫 번째 이미지 출력
plt.subplot(122)  # 두 번째 그래프 (1행 2열 중 두 번째 위치)
curr_img = np.reshape(x_test[0], (32, 32, 3))  # 첫 번째 테스트 데이터를 (32, 32, 3) 형식으로 재구성
plt.imshow(curr_img)  # 이미지를 표시
print(plt.title("(Label: " + str(label_dict[y_test[0]]) + ")"))  # 레이블 출력

### **2-2 데이터 Scaling**

In [ ]:
#pca를 위한 scaling
np.min(x_train),np.max(x_train)

In [ ]:
#scaling
x_train = x_train/255.0
np.min(x_train),np.max(x_train)

In [ ]:
#3072개의 픽셀
x_train_flat = x_train.reshape(-1,3072)
x_test_flat = x_test.reshape(-1,3072)

In [ ]:
# 특징 이름 생성 (픽셀 값에 대해 열 이름 설정)
feat_cols = ['pixel'+str(i) for i in range(x_train_flat.shape[1])]  # x_train_flat의 열 수만큼 'pixel0', 'pixel1', ... 형식의 이름 생성

# CIFAR-10 데이터를 DataFrame으로 변환
df_cifar = pd.DataFrame(x_train_flat, columns=feat_cols)  # x_train_flat 데이터를 열 이름과 함께 DataFrame으로 변환
df_cifar['label'] = y_train  # 레이블 데이터를 DataFrame의 새로운 열로 추가

# DataFrame 크기 출력
print('Size of the dataframe: {}'.format(df_cifar.shape))  # DataFrame의 크기 (행, 열) 출력

In [ ]:
df_cifar.head()

### **2-3. PCA 적용**

In [ ]:
pca_cifar = PCA(n_components=2)
principalComponents_cifar = pca_cifar.fit_transform(df_cifar.iloc[:,:-1])

In [ ]:
principal_cifar_Df = pd.DataFrame(data = principalComponents_cifar
             , columns = ['principal component 1', 'principal component 2'])
principal_cifar_Df['y'] = y_train

principal_cifar_Df.head()

In [ ]:
print('Explained variation per principal component: {}'.format(pca_cifar.explained_variance_ratio_))

In [ ]:
plt.figure(figsize=(16,10))
sns.scatterplot(
    x="principal component 1", y="principal component 2",
    hue="y",
    palette=sns.color_palette("hls", 10),
    data=principal_cifar_Df,
    legend="full",
    alpha=0.3
)


### **2-4. PCA 효과성 검증**

In [ ]:
x_test = x_test/255
x_test = x_test.reshape(-1,32,32,3)
x_test_flat = x_test.reshape(-1,3072)

In [ ]:
pca = PCA(0.9)

In [ ]:
pca.fit(x_train_flat)

In [ ]:
PCA(copy=True, iterated_power='auto', n_components=0.9, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False)

#pca수
pca.n_components_

In [ ]:
#x_train_flat: 기존 데이터
#train_img_pca: pca 적용 후 데이터
train_img_pca = pca.transform(x_train_flat)
test_img_pca = pca.transform(x_test_flat)

In [ ]:
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [ ]:
batch_size = 128
num_classes = 10
epochs = 40

In [ ]:
#원본 학습
model = Sequential()
model.add(Dense(1024, activation='relu', input_shape=(3072,)))
model.add(Dropout(0.5))  # Dropout 추가
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.5))  # Dropout 추가
model.add(Dense(512, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))
model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(),
              metrics=['accuracy'])

history = model.fit(x_train_flat, y_train,batch_size=batch_size,epochs=epochs,verbose=1,
                    validation_data=(x_test_flat, y_test))

In [ ]:
#pca 학습
model_pca = Sequential()
model_pca.add(Dense(1024, activation='relu', input_shape=(99,)))
model_pca.add(Dropout(0.5))  # Dropout 추가
model_pca.add(Dense(1024, activation='relu'))
model_pca.add(Dropout(0.5))  # Dropout 추가
model_pca.add(Dense(512, activation='relu'))
model_pca.add(Dense(256, activation='relu'))
model_pca.add(Dense(num_classes, activation='softmax'))

model_pca.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(),
              metrics=['accuracy'])

history_pca = model_pca.fit(train_img_pca, y_train,batch_size=batch_size,epochs=epochs,verbose=1,
                    validation_data=(test_img_pca, y_test))

In [ ]:
# Plotting the accuracy of the two models
plt.figure(figsize=(14, 6))

# Plot training accuracy
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Original Model - Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Original Model - Validation Accuracy')
plt.plot(history_pca.history['accuracy'], label='PCA Model - Training Accuracy')
plt.plot(history_pca.history['val_accuracy'], label='PCA Model - Validation Accuracy')
plt.title('Model Accuracy Comparison')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.tight_layout()
plt.show()